In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
import warnings 

In [3]:
warnings.filterwarnings('ignore')

In [37]:
tmp = pd.read_csv('months1_10/09.csv')
tmp = pd.concat([tmp, pd.read_csv('months1_10/10.csv')])
tmp = pd.concat([tmp, pd.read_csv('months1_10/11.csv')])

In [38]:
train = tmp.groupby(['time_stamp', 'loc_id'], as_index=False)['phone_id'].\
                                                         agg({'num':'count'})

In [40]:
train['month'] = train.time_stamp.apply(lambda x: int(x.split('-')[1]))
train['day'] = train.time_stamp.apply(lambda x: int(x.split('-')[2][:2]))
train['hour'] = train.time_stamp.apply(lambda x: int(x.split('-')[2][-2:]))
train['ymd'] = train.time_stamp.apply(lambda x: x[:-3])
train['dayofweek'] = pd.DatetimeIndex(train['ymd']).dayofweek

In [41]:
train.head(3)

,time_stamp,loc_id,num,month,day,hour,ymd,dayofweek
0,2017-09-01 00,1,136,9,1,0,2017-09-01,4
1,2017-09-01 00,2,244,9,1,0,2017-09-01,4
2,2017-09-01 00,3,45,9,1,0,2017-09-01,4


In [42]:
%%time
# 求得星期小时的均值和标准差
train = pd.merge(train, train.groupby(['loc_id', 'dayofweek', 'hour'], as_index=False)['num'].\
                agg({'week_hour_mean':'mean'}), on=['loc_id', 'dayofweek', 'hour'], how='left')
train = pd.merge(train, train.groupby(['loc_id', 'dayofweek', 'hour'], as_index=False)['num'].\
                agg({'week_hour_std':'std'}), on=['loc_id', 'dayofweek', 'hour'], how='left')

CPU times: user 88 ms, sys: 12 ms, total: 100 ms
Wall time: 97.7 ms


In [45]:
%%time
# 取2.4倍的标准差，去掉大概0.5%的异常值数据
train['del'] = train.apply(lambda x: 0 if (x['num'] <= x['week_hour_mean']+2.4*x['week_hour_std']) and \
                           (x['num'] >= x['week_hour_mean']-2.4*x['week_hour_std']) else 1,\
                            axis=1)

CPU times: user 3.64 s, sys: 0 ns, total: 3.64 s
Wall time: 3.63 s


In [49]:
# 取9月4号开学后的数据，去掉十一假期
train = train.loc[(train['ymd']>='2017-09-04')&\
                  ~((train['ymd']>='2017-10-01')&(train['ymd']<='2017-10-08'))]

In [51]:
train = train.loc[train['del']!=1]

In [53]:
del train['del']

In [ ]:
train.to_csv('data/train_9_11.csv', index=None)